In [13]:
!pip install --upgrade gradio


In [3]:
!pip install --upgrade transformers


In [4]:
!pip install --upgrade tokenizers


In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "/content/drive/MyDrive/nllb_mai_bho_finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, local_files_only=True)

model.to(device).eval()
print(" Translation model loaded successfully from Drive")


Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


 Translation model loaded successfully from Drive


In [16]:

from transformers import MBart50TokenizerFast, MBartForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
sum_model_path = "/content/drive/MyDrive/mbart_hi_en_summarizer"
tokenizer_sum = MBart50TokenizerFast.from_pretrained(sum_model_path, local_files_only=True)
model_sum = MBartForConditionalGeneration.from_pretrained(sum_model_path, local_files_only=True)
model_sum.to(device).eval()

print("Summary model loaded successfully from Drive")


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

Summary model loaded successfully from Drive


In [17]:


nllb_lang_dict = {
    "hindi": "hin_Deva",
    "english": "eng_Latn"
}

mbart_lang_dict = {
    "hindi": "hi_IN",
    "english": "en_XX"
}


def translate_text(text, lang):

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    forced_lang = nllb_lang_dict.get(lang.lower(), "hin_Deva")

    forced_token_id = tokenizer.convert_tokens_to_ids(forced_lang)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=forced_token_id,
        max_length=512,
        num_beams=4
    )

    translation = tokenizer.decode(
        outputs[0],
        skip_special_tokens=True
    )

    return translation
def summarize_text(text, target_lang):

    tgt_lang = mbart_lang_dict.get(
        target_lang.lower(),
        "hi_IN"
    )

    tokenizer_sum.src_lang = tgt_lang


    inputs = tokenizer_sum(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)


    summary_ids = model_sum.generate(
        **inputs,
        forced_bos_token_id=tokenizer_sum.lang_code_to_id[tgt_lang],
        max_length=150,
        min_length=40,
        num_beams=4
    )


    summary = tokenizer_sum.decode(
        summary_ids[0],
        skip_special_tokens=True
    )

    return summary



def pipeline(text, lang):

    translated = translate_text(text, lang)

    summary = summarize_text(translated, lang)

    return translated, summary


In [18]:
user_text = """
हम काल्हि अपन गाँव जइब। ओतऽ अपन बाबूजी सँ भेट करब आ घरक हालचाल पूछब।
खेत मे जाकऽ फसल देखब आ किसान सँ बात करब।
बाजार सँ जरूरी सामान खरीदब आ फेर दोस्त सभ सँ मिलब।
सांझ मे घर आबि परिवार संग भोजन करब आ दिनक अनुभव साझा करब।
"""

# Target Output Language
target_lang = "english"

translated_text = translate_text(user_text, target_lang)
summary_text = summarize_text(translated_text, target_lang)

print("\n========== INPUT TEXT ==========")
print(user_text)

print("\n========== TRANSLATED TEXT ==========")
print(translated_text)

print("\n========== SUMMARY ==========")
print(summary_text)


========== INPUT TEXT ==========

हम काल्हि अपन गाँव जइब। ओतऽ अपन बाबूजी सँ भेट करब आ घरक हालचाल पूछब।
खेत मे जाकऽ फसल देखब आ किसान सँ बात करब।
बाजार सँ जरूरी सामान खरीदब आ फेर दोस्त सभ सँ मिलब।
सांझ मे घर आबि परिवार संग भोजन करब आ दिनक अनुभव साझा करब।


========== TRANSLATED TEXT ==========
Tomorrow I will go to my village. There I will meet my father and ask about the house. I will go to the fields to see the crops and talk to the farmer. I will buy necessities from the market and then meet friends. I will come home in the evening to eat with the family and share the day's experiences.

========== SUMMARY ==========
Tomorrow I will go to my village to meet my father and ask about the house. Later, I will go to the fields to see the crops and talk to the farmer. Later, I will buy necessities from the market and meet friends. Later, I will come home to eat with the family and share the day's experiences.


In [19]:
!pip install -q gradio


In [20]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBart50TokenizerFast, MBartForConditionalGeneration


device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = "/content/drive/MyDrive/nllb_mai_bho_finetuned"
sum_model_path = "/content/drive/MyDrive/mbart_hi_en_summarizer"

print("Loading models...")

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    local_files_only=True,
    use_fast=False
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    local_files_only=True
).to(device).eval()

tokenizer_sum = MBart50TokenizerFast.from_pretrained(
    sum_model_path,
    local_files_only=True
)
model_sum = MBartForConditionalGeneration.from_pretrained(
    sum_model_path,
    local_files_only=True
).to(device).eval()

print(" Models loaded successfully!")
nllb_lang_dict = {
    "hindi": "hin_Deva",
    "english": "eng_Latn"
}
mbart_lang_dict = {
    "hindi": "hi_IN",
    "english": "en_XX"
}
def translate_text(text, source_lang, target_lang):

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)
    forced_lang = nllb_lang_dict.get(target_lang.lower(), "hin_Deva")
    forced_token_id = tokenizer.convert_tokens_to_ids(forced_lang)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=forced_token_id,
        max_length=512,
        num_beams=4
    )

    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation


def summarize_text(text, target_lang):

    tgt_lang = mbart_lang_dict.get(target_lang.lower(), "hi_IN")
    tokenizer_sum.src_lang = tgt_lang

    inputs = tokenizer_sum(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    summary_ids = model_sum.generate(
        **inputs,
        forced_bos_token_id=tokenizer_sum.lang_code_to_id[tgt_lang],
        max_length=150,
        min_length=40,
        num_beams=4
    )

    summary = tokenizer_sum.decode(summary_ids[0], skip_special_tokens=True)
    return summary


def full_pipeline(text, source_lang, target_lang):
    translated = translate_text(text, source_lang, target_lang)
    summary = summarize_text(translated, target_lang)
    return translated, summary



with gr.Blocks() as demo:
    gr.Markdown("##  Regional → Hindi/English Translation + Summary (NLLB + mBART)")

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Enter text (Bhojpuri or Maithili)",
                lines=8,
                placeholder="Yahan apna text paste karein..."
            )

            source_lang = gr.Dropdown(
                choices=["Bhojpuri", "Maithili"],
                label="Source Language"
            )

            target_lang = gr.Dropdown(
                choices=["Hindi", "English"],
                label="Output Language"
            )

            btn = gr.Button("Translate & Summarize 🚀")

        with gr.Column():
            translated_output = gr.Textbox(label="Translated Text", lines=6)
            summary_output = gr.Textbox(label="Summary", lines=6)

    btn.click(
        full_pipeline,
        inputs=[input_text, source_lang, target_lang],
        outputs=[translated_output, summary_output]
    )

demo.launch(debug=True, share=True)


Loading models...


Loading weights:   0%|          | 0/512 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Loading weights:   0%|          | 0/516 [00:00<?, ?it/s]

 Models loaded successfully!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1b4e41948b16d1998c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1b4e41948b16d1998c.gradio.live
